In [1]:
import numpy as np
import pandas as pd
from river import optim
from river.drift import ADWIN
from ray.tune.sklearn import TuneGridSearchCV
from sail.models.river.forest import AdaptiveRandomForestRegressor
from sail.models.river.linear_model import LinearRegression
from sail.models.auto_ml import PipeLineSelection
from sail.pipeline import SAILPipeline
from sklearn.impute import SimpleImputer
from sail.transfomers.river.preprocessing import StandardScaler

#### Load Data

In [2]:
df = pd.read_csv("../../datasets/HDWF2.csv")
X = df.copy()

y = X["power"]
X.drop("power", axis=1, inplace=True)
X.drop("time", axis=1, inplace=True)

#### Model Definition

In [3]:
linear_reg = LinearRegression(optimizer=optim.SGD(0.1))
random_forest = AdaptiveRandomForestRegressor(n_models=10)

#### Create SAIL Pipeline

In [4]:
steps = [
    ("Imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
    ("standard_scalar", StandardScaler()),
    ("classifier", "passthrough"),
]
sail_pipeline = SAILPipeline(steps=steps)

[2023-05-12 12:50:22:685] - INFO - SAIL : created SAILPipeline object with ID 062969ca-fa62-4b7d-8ff9-ae9a58aa8016


#### HPO Params

In [5]:
param_grid = [
    {
        "classifier": [linear_reg],
    },
    {
        "classifier": [random_forest],
    },
]

#### Create Model Instance

In [6]:
pipeline_selection = PipeLineSelection(
    pipeline=sail_pipeline,
    pipeline_params_grid=param_grid,
    scoring="R2",
    search_data_size=1000,
    search_method=TuneGridSearchCV,
    search_method_params={
        "max_iters": 1,
        "early_stopping": False,
        "scoring": "r2",
        "pipeline_auto_early_stop": False,
    },
    drift_detector=ADWIN(delta=0.001),
)

#### Start Training

In [7]:
y_preds = []
y_true = []
batch_size = 50

start = 0
for end in range(50, 15351, batch_size):

    X_train = X.iloc[start:end]
    y_train = y.iloc[start:end]

    if end > 1000: # search_data_size is 1000
        preds = pipeline_selection.predict(X_train)
        y_preds.extend(list(preds))
        y_true.extend(list(y_train))

    pipeline_selection.partial_fit(X_train, y_train)
    start = end

2023-05-12 12:50:30,081	WARNING searcher.py:372 -- save not implemented for Searcher. Skipping save.
(_Trainable pid=32193) [2023-05-12 12:50:34:662] - INFO - SAIL : created SAILPipeline object with ID b2147b11-c371-49c6-891a-313f88b71a39
(_Trainable pid=32193) [2023-05-12 12:50:34:663] - INFO - SAIL : created SAILPipeline object with ID d17d390a-59d6-4ec7-ac25-7bd2d18f14a1
(_Trainable pid=32193) [2023-05-12 12:50:34:664] - INFO - SAIL : created SAILPipeline object with ID a603d407-882c-4f13-a2ff-6ceaf3ccaff2
(_Trainable pid=32193) [2023-05-12 12:50:34:664] - INFO - SAIL : created SAILPipeline object with ID d916ca67-439b-4567-8809-3d31f31d8a4e
(_Trainable pid=32193) [2023-05-12 12:50:34:665] - INFO - SAIL : created SAILPipeline object with ID f037fdf9-2c71-4e7d-930d-28d76bc7dcdc
(_Trainable pid=32193) [2023-05-12 12:50:34:675] - INFO - SAIL : created SAILPipeline object with ID d22ffcf3-b16f-4cd7-9e26-989d4f65da45
(_Trainable pid=32193) [2023-05-12 12:50:34:729] - INFO - SAIL : create

(_Trainable pid=32193) [Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
(_Trainable pid=32193) [Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
(_Trainable pid=32193) [Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
(_Trainable pid=32193) [Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
(_Trainable pid=32193) [Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.0s
(_Trainable pid=32194) [Pipeline] ........ (step 3 of 3) Processing classifier, total=   2.7s


(_Trainable pid=32194) [2023-05-12 12:50:37:523] - INFO - SAIL : created SAILPipeline object with ID 63140f1f-0953-4b1b-9de3-2fcc93504939


(_Trainable pid=32194) [Pipeline] ........ (step 3 of 3) Processing classifier, total=   2.8s


(_Trainable pid=32194) [2023-05-12 12:50:40:374] - INFO - SAIL : created SAILPipeline object with ID e7efe7c6-ddb9-4f02-a81e-02ce53a7ac97


...

[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.4s


[2023-05-12 12:52:36:52] - INFO - SAIL : Cumulative Pipeline Score: 0.654327381682971
[2023-05-12 12:52:36:53] - INFO - SAIL : Calling Partial_fit() on the pipeline.


[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.3s


[2023-05-12 12:52:36:401] - INFO - SAIL : Cumulative Pipeline Score: 0.6543356719464379
[2023-05-12 12:52:36:402] - INFO - SAIL : Calling Partial_fit() on the pipeline.


[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.3s
[Pipeline] ........ (step 3 of 3) Processing classifier, total=   0.3s


#### Final Score

In [9]:
pipeline_selection.cumulative_score

0.6543356719464379

In [11]:
import plotly.express as px

df = pd.DataFrame({"y_true": y_true, "y_preds": y_preds}).head(500)
fig = px.line(df, y=["y_true", "y_preds"], title='')
fig.show()